In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import lxml

In [ ]:
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options, service=ChromeService(ChromeDriverManager().install()))
driver.implicitly_wait(10)

In [ ]:
base_athlete_list = []
athlete_list = []

for x in range(1,12):
    currPage = f'https://worldathletics.org/world-rankings/1500m/men?regionType=world&page={x}&rankDate=2022-08-09&limitByCountry=0'

    # so can get max number of pages
    try:
        driver.get(currPage)
        driver.implicitly_wait(10)
    except:
        break

    page = driver.page_source

    sp = BeautifulSoup(page, 'lxml') #lxml is faster than python's built in parser

    trTags = sp.select('table.records-table > tbody > tr')
    for trTag in trTags:
        # Get these details now before the DOM changes
        name = trTag.select_one('td[data-th="Competitor"]').text.strip().replace('\n','')
        dob = trTag.select_one('td[data-th="DOB"]').text.strip().replace('\n','')
        nat = trTag.select_one('td[data-th="Nat"]').text.strip().replace('\n','')
        score = int(trTag.select_one('td[data-th="score"]').text.strip().replace('\n',''))
        events = trTag.select_one('td[data-th="EventList"]').text.strip().replace('\n','')

        # some of the links wouldn't work because the data-athlete-url had a 0 in it
        link = 'https://worldathletics.org' + (trTag.attrs['data-athlete-url'].replace('-0','-'))
        
        base_athlete_item = {
            'name': name,
            'dob': dob,
            'nat': nat,
            'score': score,
            'events': events,
            'link': link,
        }
        base_athlete_list.append(base_athlete_item)
    
    # Go to each athletes page and scrape their progression charts
    for y in base_athlete_list:
        try:
            driver.get(y['link'])
            driver.implicitly_wait(10)
            driver.execute_script(
                "arguments[0].click();",
                driver.find_elements(
                        By.CSS_SELECTOR,
                        'div.profileStatistics_tab__1Blal'
                )[4]
            )
            time.sleep(1)
            page = driver.page_source
        except:
            print("no selection")
            continue

        sp = BeautifulSoup(page, 'lxml')
        wholeTable = sp.select_one('div.profileStatistics_tabContent__vINmY.profileStatistics_active__1QQ9F')
        tableDivs = wholeTable.select('div.profileStatistics_statsTable__xU9PN')
        for div in tableDivs:
            k = div.select('div.profileStatistics_tableName__2qDVZ')
            environment = k[0].text
            race = k[1].text
            trs = div.select('tbody.profileStatistics_tableBody__1w5O9 > tr')
            for tr in trs:
                performance = tr.select_one('td:nth-child(2)').text
                place = tr.select_one('td:nth-child(3)').text
                date = tr.select_one('td:nth-child(4)').text
                athlete_item = {
                    'name': y['name'],
                    'dob': y['dob'],
                    'nat': y['nat'],
                    'score': y['score'],
                    'events': y['events'],
                    'environment': environment,
                    'race': race,
                    'date': date,
                    'performance': performance,
                    'place': place
                }
                athlete_list.append(athlete_item)
                print(athlete_item)
    # Chunk the data per page into individual csv files
    print(f'page {x} done')
    pd.DataFrame(athlete_list).to_csv(f"pg_{x}.csv")
    athlete_list = []
    base_athlete_list = []

    # Took 65m 9.9s to run this code against 10 pages of rankings